In [2]:
import pickle
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from pymongo import MongoClient
from pymongo.errors import BulkWriteError, DuplicateKeyError
import pandas_datareader as pdr
import numpy as np
import quandl
import nltk
import pandas_datareader as pdr
import itertools
import torch
import torch.nn as nn
import spacy
from gensim.models.word2vec import  Word2Vec
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import TaggedLineDocument
import collections

In [3]:
lmap=lambda fun,it:list(map(lambda x:fun(x),it))

In [4]:
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe(nlp.create_pipe('sentencizer'))
n = lambda x: nlp(x, disable=['tagger', 'ner', 'textcat', 'parser'])

In [10]:
titles=titles.sort_values('date')
titles.index = range(titles.shape[0])

In [12]:
titles=pd.read_csv('data/title_text.csv')

In [14]:
titles=titles.drop_duplicates()

In [13]:
titles=titles.sort_values('date')
titles.index = range(titles.shape[0])

In [11]:
titles.to_csv('data/title_text.csv')

In [11]:
title_text=titles['title']

In [12]:
mean_titles=[]
for title in tqdm_notebook(list(title_text)):
    title_=n(title)
    mean_titles.append(title_.vector)

HBox(children=(IntProgress(value=0, max=442666), HTML(value='')))

In [16]:
weight_matrix=[]
word_index = collections.defaultdict()
word_index.setdefault('', len(word_index))
weight_matrix.append(np.zeros(300,))

word_index.setdefault('<GO>',len(word_index))
weight_matrix.append(np.ones(300,))

word_index.setdefault('<EOS>', len(word_index))
weight_matrix.append(-np.ones(300,))

indexed_titles=[]
for title in tqdm_notebook(list(title_text)):
    title_=n(title)
    title_tokens = list(title_)
    indexed_title=[]
    indexed_title.append(word_index['<GO>'])
    for t in title_tokens:
        t_text = t.text
        if t.is_oov:
            t.vocab.set_vector(t_text, np.random.uniform(-1, 1, (300,)))
        if t_text not in word_index:
            word_index.setdefault(t_text, len(word_index))
            weight_matrix.append(t.vector)
        indexed_title.append(word_index[t_text])
    indexed_title.append(word_index['<EOS>'])
    indexed_titles.append(indexed_title)

HBox(children=(IntProgress(value=0, max=442666), HTML(value='')))

In [17]:
weight_matrix=np.array(weight_matrix)

In [18]:
max_length=max(lmap(lambda x:len(x),indexed_titles))

In [19]:
def crop_pad(max_leng, word_index):
    if len(word_index) > max_leng:
        return word_index[:max_leng]
    pad_leng = max_leng - len(word_index)
    word_index = word_index + [0] * pad_leng
    assert len(word_index) == max_leng
    return word_index

In [20]:
padded_titles=lmap(lambda x:crop_pad(max_length,x),indexed_titles)

In [21]:
corpus=np.array(padded_titles)

In [22]:
groups=titles.groupby('date')

In [36]:
mean_titles=titles.drop('title',axis=1).join(pd.DataFrame(mean_titles)).groupby('date').mean().dropna()

In [37]:
mean_titles.to_csv('../DLQuant/data/mean_titles.csv')

In [38]:
np.save('../DLQuant/data/title_weight_matrix',weight_matrix)

In [39]:
np.save('../DLQuant/data/title_sequence',corpus)

In [47]:
np.save('../DLQuant/output/RNNAE/date_index',titles.date.values)